# Evaluate 
$ f(x, y) = 2x^2 - x^2y^2 + 3 $

$ g_1 = x ⋅ x $ 

$ g_2 = g1 ⋅ 2 $

$ g_3 = y ⋅ y $

$ g_4 = g1 ⋅ g3 $

$ g_5 = g2 - g4 $

$ g_6 = g5 + 3 $

---

$ c_1 = a_1 ⋅ b_1 $

$ c_2 = a_2 ⋅ 2 $

$ c_3 = a_3 ⋅ b_3 $

$ c_4 = a_4 ⋅ b_4 $

$ c_5 = a_5 - b_5 $

$ c_6 = a_6 + 3 $

---

$ x = a_1 = b_1 $

$ y = a_3 = b_3 $

$ g_1 = a_2 = c_1 $

$ g_2 = a_5 = c_2 $

$ g_3 = b_4 = c_3 $

$ g_4 = b_5 = c_4 $

$ g_5 = a_6 = c_5 $

$ g_6 = c_6 $

### Gate constrain equation

$ g_i = q_{Li}⋅a_i+q_{Ri}⋅b_i+q_{Mi}⋅(a_i⋅b_i)+q_{Ci}+q_{Oi}⋅c_i=0 $

$ g_1 = 0⋅a_1 + 0⋅b_1 + 1⋅(a_1⋅b_1) + 0 + (-1)⋅c_1 = 0 $

$ g_2 = 2⋅a_2 + 0⋅b_2 + 0⋅(a_2⋅b_2) + 0 + (-1)⋅c_2 = 0 $

$ g_3 = 0⋅a_3 + 0⋅b_3 + 1⋅(a_3⋅b_3) + 0 + (-1)⋅c_3 = 0 $

$ g_4 = 0⋅a_4 + 0⋅b_4 + 1⋅(a_4⋅b_4) + 0 + (-1)⋅c_4 = 0 $

$ g_5 = 1⋅a_5 + (-1)⋅b_5 + 0⋅(a_5⋅b_5) + 0 + (-1)⋅c_5 = 0 $

$ g_6 = 1⋅a_6 + 0⋅b_6 + 0⋅(a_6⋅b_6) + 3 + (-1)⋅c_6 = 0 $

**selector vectors**

$ g_i = [q_{Li}, q_{Ri}, q_{Mi}, q_{Ci}, q_{Oi}] $

$ g_1 = [0, 0, 1, 0, -1] $

$ g_2 = [2, 0, 0, 0, -1] $

$ g_3 = [0, 0, 1, 0, -1] $

$ g_4 = [0, 0, 1, 0, -1] $

$ g_5 = [1, -1, 0, 0, -1] $

$ g_6 = [1, 0, 0, 3, -1] $

**let x = 2 y = 3**


| Vectors| a | b | c | qLi | qRi | qMi | qCi | qOi |
| --- | --- | --- | --- |--- | --- | --- | --- | --- |
| gate1 | 2 | 2 | 4 | 0 | 0 | 1 | 0 | -1 |
| gate2 | 4 | 0 | 8 | 2 | 0 | 0 | 0 | -1 |
| gate3 | 3 | 3 | 9 | 0 | 0 | 1 | 0 | -1 |
| gate4 | 4 | 9 | 36 | 0 | 0 | 1 | 0 | -1 |
| gate5 | 8 | 36 | -28 | 1 | -1 | 0 | 0 | -1 |
| gate6 | -28 | 3 | -25 | 1 | 0 | 0 | 3 | -1 |

In [2]:
import galois

x = 2
y = 3

out = 2*x**2 - x**2*y**2 + 3
print(out)

a = [2, 4, 3, 4, 8, -28]
b = [2, 0, 3, 9, 36, 3]
c = [4, 8, 9, 36, -28, -25]

ql = [0, 2, 0, 0, 1, 1]
qr = [0, 0, 0, 0, -1, 0]
qm = [1, 0, 1, 1, 0, 0]
qc = [0, 0, 0, 0, 0, 3]
qo = [-1, -1, -1, -1, -1, -1]

def to_galois_array(a, field):
    # normalize to positive values
    a = [x % field.order for x in a]
    return GF(a)

def to_vanishing_poly(a, field):
    p = galois.Poly([1, -int(a[0]) % field.order], field=field)
    for i in range(1, len(a)):
        p *= galois.Poly([1, -int(a[i]) % field.order], field=field)
    return p

def to_poly(x, v, field):
    assert len(x) == len(v)
    y = to_galois_array(v, field)
    return galois.lagrange_poly(x, y)

p = 73
GF = galois.GF(p)
w = GF(6)

roots = GF([w**x for x in range(0, len(a))])

_A = to_poly(roots, a, GF)
_B = to_poly(roots, b, GF)
_C = to_poly(roots, c, GF)
Z = to_vanishing_poly(roots, GF)

bA = galois.Poly.Random(degree=1, field=GF)
bB = galois.Poly.Random(degree=1, field=GF)
bC = galois.Poly.Random(degree=1, field=GF)

A = _A + bA*Z
B = _B + bB*Z
C = _C + bC*Z

QL = to_poly(roots, ql, GF)
QR = to_poly(roots, qr, GF)
QM = to_poly(roots, qm, GF)
QC = to_poly(roots, qc, GF)
QO = to_poly(roots, qo, GF)

# gate constraints polynomial
# g(x) = a(x)*ql(x) + b(x)*qr(x) + a(x)*b(x)*qm(x) + qc(x) + c(x)*qo(x)
G = A*QL + B*QR + A*B*QM + QC + C*QO

T = G // Z

assert G % Z == 0

print("--- Vanishing Polynomial ---")
print("Z = ", Z)

print("\n\n--- Blinding Polynomials ---")
print(f"bA = {bA}")
print(f"bB = {bB}")
print(f"bC = {bC}")

print("\n\n--- Wire Polynomials ---")
print(f"A = {_A}")
print(f"B = {_B}")
print(f"C = {_C}")

print("\n\n--- Blinded Wire Polynomials ---")
print(f"A' = {A}")
print(f"B' = {B}")
print(f"C' = {C}")

print("\n\n--- Selector Polynomials ---")
print(f"QL = {QL}")
print(f"QR = {QR}")
print(f"QM = {QM}")
print(f"QC = {QC}")
print(f"QO = {QO}")

print("\n\n--- Gate Constraints Polynomial ---")
print(f"G = {G}")
for i in range(0, len(roots)):
    print(f"gate #{i+1} G({roots[i]}) = {G(roots[i])} --> {'OK' if G(roots[i]) == 0 else 'FAIL'}")

print("\n\n--- Target Polynomial ---")
print(f"T = {T}")

-25


OMP: Info #270: omp_set_nested routine deprecated, please use omp_set_max_active_levels instead.


--- Vanishing Polynomial ---
Z =  x^6 + 13x^5 + 26x^4 + 7x^3 + 39x^2 + 11x + 49


--- Blinding Polynomials ---
bA = 24x + 63
bB = 29x + 31
bC = 4x + 47


--- Wire Polynomials ---
A = x^5 + 18x^4 + 14x^3 + 45x^2 + 35x + 35
B = 11x^5 + 35x^4 + 23x^3 + 23x^2 + 40x + 16
C = 50x^5 + 8x^4 + 72x^3 + 58x^2 + 72x + 36


--- Blinded Wire Polynomials ---
A' = 24x^7 + 10x^6 + 57x^5 + 72x^4 + 4x^3 + 65x^2 + 6x + 56
B' = 29x^7 + 43x^6 + 22x^4 + 57x^3 + 18x^2 + 50x + 2
C' = 4x^7 + 26x^6 + 35x^5 + 17x^4 + 46x^3 + 37x^2 + 55x + 3


--- Selector Polynomials ---
QL = 69x^5 + 65x^4 + 3x^3 + 50x^2 + 17x + 15
QR = 35x^5 + 44x^4 + 45x^3 + 19x^2 + x + 2
QM = 27x^5 + 7x^4 + 5x^3 + 13x^2 + 17x + 5
QC = 28x^5 + 41x^4 + 23x^3 + 48x^2 + 69x + 10
QO = 72


--- Gate Constraints Polynomial ---
G = 31x^19 + 51x^18 + 63x^17 + 24x^16 + 29x^15 + 2x^14 + 29x^13 + 46x^12 + 61x^11 + 40x^10 + 63x^9 + 41x^8 + 55x^7 + 70x^6 + 5x^5 + 67x^4 + 44x^3 + 18x^2 + 40x + 24
gate #1 G(1) = 0 --> OK
gate #2 G(6) = 0 --> OK
gate #3 G(36) 

In [10]:
from py_ecc.optimized_bls12_381 import G1, multiply, add

# powers of tau
tau = 45

powers_of_tau = [multiply(G1, tau**i) for i in range(0, G.degree)]

def msm(powers, poly: galois.Poly):
    coeff = poly.coeffs[::-1]
    res = multiply(G1, 0)
    for i in range(0, len(coeff)):
        res = add(res, multiply(powers[i], int(coeff[i])))

    return res

A_commitment = msm(powers_of_tau, A)
B_commitment = msm(powers_of_tau, B)
C_commitment = msm(powers_of_tau, C)
T_commitment = msm(powers_of_tau, T)

print("\n\n--- Commitments ---")
print(f"A_commitment = {A_commitment}")
print(f"B_commitment = {B_commitment}")
print(f"C_commitment = {C_commitment}")
print(f"T_commitment = {T_commitment}")



--- Commitments ---
A_commitment = (1464410537002715019391287920272217529160431373257921755567194026176923347493254545872373899531762403839000598212140, 3060382092429178157846218111083241346830436483315356214383418030294221250493589745521356819371844807656141147761721, 1728872800908733786248989550208146391873904675704517030581674105539642455628775752748334550386340552301963305980109)
B_commitment = (1345571991936491126046849656511720135403465145805456625065736374642728536429855990952951630653847989645792194954756, 2866098195987794353146069009530792333962930870007463896858854883037469416779185026449328420322040804116839975070932, 1314024039297981441181762083168139126165191735287546920161846707429687006209576892495699431454207417956996772112727)
C_commitment = (1838794999348323006472662694453160334112322469842343552798609000491803539619985780796236425805028493407091234159383, 213096798301924388460865708767286469693180109914276459384392380928932872118773892183785028744994348014271612887

### Permutation constraint

In [21]:
_a = to_galois_array(a, GF)
a_map = { 0: 0, 1: 3, 2: 2, 3: 1, 4: 4, 5: 5}

a_prime = [(x, roots[i]) for i, x in enumerate(_a)]
a_prime_prime = [(x, roots[a_map[i]]) for i, x in enumerate(_a)]

print("\n\n--- Wire Values ---")

beta = GF(2)
gamma = GF(3)
a_p = [x + roots[i]*beta + gamma for i, x in enumerate(_a)]
a_pp = [x + roots[a_map[i]]*beta + gamma for i, x in enumerate(_a)]

print(f"a = {_a}")
print(f"a' = {a_p}")
print(f"a'' = {a_pp}")

_a_p = [A(roots[i]) + beta*roots[i] + gamma for i in range(0, len(roots))]
_a_pp = [A(roots[a_map[i]]) + beta*roots[a_map[i]] + gamma for i in range(0, len(roots))]

print(f"A' = {_a_p}")
print(f"A'' = {_a_pp}")

# multiply all elements in a list
def prod(a):
    res = 1
    for x in a:
        res *= x
    return res

assert prod(a_p) == prod(a_pp)
assert prod(_a_p) == prod(_a_pp)



--- Wire Values ---
a = [ 2  4  3  4  8 45]
a' = [GF(7, order=73), GF(19, order=73), GF(5, order=73), GF(1, order=73), GF(48, order=73), GF(51, order=73)]
a'' = [GF(7, order=73), GF(1, order=73), GF(5, order=73), GF(19, order=73), GF(48, order=73), GF(51, order=73)]
A' = [GF(7, order=73), GF(19, order=73), GF(5, order=73), GF(1, order=73), GF(48, order=73), GF(51, order=73)]
A'' = [GF(7, order=73), GF(1, order=73), GF(5, order=73), GF(19, order=73), GF(48, order=73), GF(51, order=73)]
